<a href="https://colab.research.google.com/github/madeirabelo/huggingFace/blob/main/ler_epub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ebooklib
!pip install PyDrive


     |████████████████████████████████| 111 kB 25.8 MB/s 
  Created wheel for ebooklib: filename=EbookLib-0.17.1-py3-none-any.whl size=38183 sha256=59ed1b1a68475f4a5560fdb755345876f48a8d6670e15683375e46baf812ef00
  Stored in directory: /root/.cache/pip/wheels/43/39/fd/db4f652431a55d28472ba7f5f7c9a8efad03b97f443a48ea2f
Successfully built ebooklib


In [ ]:
#Para escrever um ficheiro
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
#drive.CreateFile({'parents': [{'id': '### folder ID ###'}]})
drive = GoogleDrive(gauth)
file_epub = drive.CreateFile({'title': 'Neal_Stephenson_-_Termination_Shock.txt', 'parents' : [{'id':'1FSJ6h9LNqE270aY8tCDBEFEZ3D06rd8Z'}]})  # Create GoogleDriveFile instance with title 'Hello.txt'.
file_epub.SetContentString('Hello World 1!') # Set content of the file from given string.
file_epub.GetContentFile('Neal_Stephenson_-_Termination_Shock.txt')
update = file_epub.GetContentString() + "\ntest"
file_epub.SetContentString(update)
file_epub.Upload()

In [ ]:
import re
from bs4 import BeautifulSoup
from gensim.utils import tokenize

from ebooklib import epub
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Authorize with your Google ID, and paste in the link that comes up and press Enter!
downloaded = drive.CreateFile({'id':"1Fq-C5HTqxh97aXW5YTGo4ZRrrIa1zI7i"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Neal_Stephenson_-_Termination_Shock.epub')        # replace the file name with your file
f = open('Neal_Stephenson_-_Termination_Shock.epub', 'r+b')
book = epub.read_epub(f)
ITEM_UNKNOWN      = 0
ITEM_IMAGE        = 1
ITEM_STYLE        = 2
ITEM_SCRIPT       = 3
ITEM_NAVIGATION   = 4
ITEM_VECTOR       = 5
ITEM_FONT         = 6
ITEM_VIDEO        = 7
ITEM_AUDIO        = 8
ITEM_DOCUMENT     = 9
ITEM_COVER        = 10

#print("000000000000000000000000000000")
paragraphs = []
capitulo_livro = [0]
capitulo = -1
content = [BeautifulSoup(x.content, 'html.parser') for x in book.get_items_of_type(ITEM_DOCUMENT)]

for c in content:
    #print("1111111111111111111111111111111")
    #print(c)
    capitulo = capitulo + 1
    #paragraph_parse_tags = ['p', 'ol', 'ul']
    paragraph_parse_tags = ['p', 'ol_', 'ul_']
    #paragraphs += f"capitulo={capitulo}"
    old_capitulo = 0 
    capitulo_rch = 0
    paragraphs = []
    for tag in paragraph_parse_tags:       
        for element in c.find_all(tag):
            if element.a != None:
              if "#rch" in element.a['href']:
                rch = re.search("(\d*$)",element.a['href'])
                capitulo_rch = int(rch.group(1))           # capitulo visto no texto
            #try:
              #if element.name != 'nav':
            #print(element)   #DEBUG            
            text = element.get_text(' ', strip=True).encode('ascii', "ignore")
            text_str = '\n' + text.decode('UTF-8')
            try:
              if element.span['class'] == ['dropcap']:
                text_str = text_str.replace(" ", "", 1)    
            except:
              pass
            paragraphs += [text_str] 
            #print(text_str)  #DEBUG 
            #except:
            #    pass
    if capitulo_rch != 0:           
      capitulo_livro.insert(capitulo_rch, paragraphs)
drive = GoogleDrive(gauth)
file_epub = drive.CreateFile({'title': 'Neal_Stephenson_-_Termination_Shock.txt', 'parents' : [{'id':'1FSJ6h9LNqE270aY8tCDBEFEZ3D06rd8Z'}]})  # Create GoogleDriveFile instance with title 'Hello.txt'.
file_epub.SetContentString("Neal Stephenson - Termination Shock") # Set content of the file from given string.
for p in capitulo_livro:
  if p != 0:
    #print (p)
    #file_epub.SetContentString(p) # Set content of the file from given string.
    file_epub.GetContentFile('Neal_Stephenson_-_Termination_Shock.txt')
    update = file_epub.GetContentString() + '\n' + ' '.join(p)
    file_epub.SetContentString(update)
    file_epub.Upload()

 
######################################
# TODO alimentar o sistema de resumo 
######################################


  #print(content)
  #break
  #if item.get_type() == ITEM_DOCUMENT :
  #    print('==================================')
  #    print('NAME : ', item.get_name())
  #    print('----------------------------------')
  #    print(item.get_content())
      #print('==================================')

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))